# Gathering Data

Importing requried librarys

In [12]:
%%writefile ../src/datautil.py
#datautil.py
import os
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

Overwriting ../src/datautil.py


Functions for Downloading data

In [13]:
%%writefile -a ../src/datautil.py

def get_soup(url):
    
    
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    return soup

def download_data(url, n=50, sleep=1):
    cpus = [dict() for x in range(n+1)]
    soup = get_soup(url)
    table = soup.find('div', class_='chart')
    rows = table.find_all('li')
    attributes = ['Socket', 'Clocks', 'Turbo ', 'Cores:', 'Typica']

    i = 0
    for row in rows:
        link = row.find('a').get('href')
        link = 'https://www.cpubenchmark.net/' + link

        soup = get_soup(link)
        price = row.find('span', class_='price-neww').get_text()
        name = row.find('span', class_='prdname').get_text()
        

        cpus[i]["name"] = name
        cpus[i]["price"] = price

        table = soup.find('div', class_= 'left-desc-cpu')
        body = table.find_all('p')
        for col in body:
            text = col.get_text()
            title = col.find('strong').get_text()

            #Verifying attribute is one that we want
            if title[:6] in attributes:
                data = text.split(':')

                #Edge case for cores/threads being in the same exact <p> tag
                if title == "Cores:":

                    #Another edge case for amd cpu's
                    if (text.find("(") != -1):
                        data[1] = data[1][1]
                        data.append("Threads")
                        data.append(data[1])

                    else:
                        temp2 = data[2]
                        temp1 = data[1]
                        data[1] = data[1][1]
                        data[2] = temp1[3:]
                        data.append(temp2[1:])

                    cpus[i][data[2]] = data[3]

                ##Formatting strings
                if title != "Socket:":
                    data[1] = data[1].replace(" ", "")
                else:
                    data[1] = data[1][1:]

                if (title=='Clockspeed:') or (title=='Turbo Speed:'):
                    data[1] = data[1][:-3]

                #Assign to dict
                cpus[i][data[0]] = data[1]

        if i >= n-1:
            break
        i = i+1
        time.sleep(sleep)
    
    return pd.DataFrame.from_dict(cpus)

Appending to ../src/datautil.py


Adding local src folder to sys.path

In [14]:
import sys
project_dir = '/home/atoris/course-project-thomas-wright/src'
if project_dir not in sys.path:
    sys.path.insert(0, project_dir)

print(sys.path)

['/usr/home/atoris/course-project-thomas-wright/notebooks', '/usr/local/lib/python37.zip', '/usr/local/lib/python3.7', '/usr/local/lib/python3.7/lib-dynload', '', '/usr/local/lib/python3.7/site-packages', '/usr/local/lib/python3.7/site-packages/IPython/extensions', '/usr/home/atoris/.ipython', '/home/atoris/course-project-thomas-wright/src']


In [15]:
import os
import pandas as pd
import datautil as du
import importlib

url = 'https://www.cpubenchmark.net/mid_range_cpus.html'

df = du.download_data(url, 1, 1)


#### Verify the DataFrame

In [16]:
df.head(5)

,name,price,Socket,Clockspeed,Turbo Speed,Threads,Cores,Typical TDP
0,Intel Core i5-6600 @ 3.30GHz,$262.13*,LGA 1151,3.3,3.9,4,4,65W
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Functions for saving and reloading the dataframe

In [17]:
%%writefile -a ../src/datautil.py

def save_df(df, filename):
    basedir = os.path.dirname(os.getcwd())
    filepath = 'data/' + filename
    full_path = os.path.join(basedir, filepath)
    df.to_csv(full_path, index=False)

def load_data(url, file):
    file = os.path.dirname(os.getcwd()) + '/data/' + file
    print(file)
    if not os.path.exists(file):
        if not os.path.exists(os.path.dirname(file)):
            os.makedirs(os.path.dirname(file))
        df = download_data(url)
        save_df(df, file)
    return pd.read_csv(file)


Appending to ../src/datautil.py


Use these functions to save our data

In [18]:
importlib.reload(du)

du.save_df(df, 'cpu_data.csv')

Verify new project files have been saved

In [19]:
!tree ../

../
├── LICENSE
├── README.md
├── data
│   └── cpu_data.csv
├── models
├── notebooks
│   ├── 0_setup_project_folders.ipynb
│   ├── 1_Retrieving_data.ipynb
│   ├── 2_Cleaning_data.ipynb
│   └── __pycache__
│       └── datautil.cpython-37.pyc
├── report
│   ├── interim.md
│   ├── proposal.md
│   └── report.md
└── src
    ├── __pycache__
    │   └── datautil.cpython-37.pyc
    └── datautil.py

7 directories, 12 files


Next step is to clean the rest of the data